In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import os
from PIL import Image
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
#from linformer import Linformer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%pip install einops

In [ ]:
%cd drive/MyDrive/Colab Notebooks/ViT

/content/drive/MyDrive/Colab Notebooks/ViT


In [ ]:
from vit import ViT

In [ ]:
import wandb

# 1. Start a new run
wandb.init(project='VisitonTransformer', entity='nogunogu')
# 2. Save model inputs and hyperparameters
config = wandb.config
config.learning_rate = 0.01

In [ ]:
import torch.backends.cudnn as cudnn

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_cuda = torch.cuda.is_available()
cudnn.benchmark = True
print('Use CUDA:', use_cuda)

Use CUDA: True


In [ ]:
# Training settings
batch_size = 64
epochs = 20
lr = 3e-5
gamma = 0.7
seed = 42

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed)

In [ ]:
train_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()])

val_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()])

test_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()])

In [ ]:
trainset = datasets.CIFAR10(root='./data',
                            train=True,
                            download=False, 
                            transform=train_transforms)

trainloader = DataLoader(trainset,
                         batch_size=batch_size,
                         shuffle=True,
                         num_workers=2)

testset = datasets.CIFAR10(root='./data',
                           train=False,
                           download=False,
                           transform=val_transforms)

testloader = DataLoader(testset,
                        batch_size=batch_size,
                        shuffle=False,
                        num_workers=2)

In [ ]:
model = ViT(
    dim=128,
    image_size=224,
    patch_size=32,
    num_classes=10,
    #transformer=efficient_transformer,
    channels=3,
    depth=12,
    heads=8,
    mlp_dim = 2048
).to(device)

In [ ]:
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)
# scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

In [ ]:
# 3. Log gradients and model parameters
wandb.watch(model)
for epoch in range(epochs):
    epoch_loss = 0
    epoch_accuracy = 0

    for data, label in tqdm(trainloader):
        data = data.to(device)
        label = label.to(device)

        output = model(data)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc / len(trainloader)
        epoch_loss += loss / len(trainloader)

    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for data, label in testloader:
            data = data.to(device)
            label = label.to(device)

            val_output = model(data)
            val_loss = criterion(val_output, label)

            acc = (val_output.argmax(dim=1) == label).float().mean()
            epoch_val_accuracy += acc / len(testloader)
            epoch_val_loss += val_loss / len(testloader)

    print(
        f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
    )

    # 4. Log metrics to visualize performance
    wandb.log({"loss": epoch_loss:.4f})


Epoch : 1 - loss : 2.0857 - acc: 0.2083 - val_loss : 1.9736 - val_acc: 0.2665




Epoch : 2 - loss : 1.9152 - acc: 0.2901 - val_loss : 1.8167 - val_acc: 0.3405




Epoch : 3 - loss : 1.7557 - acc: 0.3612 - val_loss : 1.6564 - val_acc: 0.3972




Epoch : 4 - loss : 1.6385 - acc: 0.4074 - val_loss : 1.5995 - val_acc: 0.4273




Epoch : 5 - loss : 1.5656 - acc: 0.4363 - val_loss : 1.5343 - val_acc: 0.4490



...


Epoch : 16 - loss : 1.2313 - acc: 0.5632 - val_loss : 1.2471 - val_acc: 0.5546




Epoch : 17 - loss : 1.2132 - acc: 0.5697 - val_loss : 1.2396 - val_acc: 0.5654




Epoch : 18 - loss : 1.2022 - acc: 0.5734 - val_loss : 1.2451 - val_acc: 0.5542




Epoch : 19 - loss : 1.1870 - acc: 0.5801 - val_loss : 1.2090 - val_acc: 0.5676




Epoch : 20 - loss : 1.1707 - acc: 0.5868 - val_loss : 1.2106 - val_acc: 0.5747



In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Apr 23 06:39:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    44W / 250W |   2451MiB / 16280MiB |     25%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------